# Wedge Task 1- sending to GBQ

I am going to B route, so these files are all already clean. This has been a challenge! I actually had to, at full disclosure had to update the code for every file because I couldn't get it to work otherwise. Although its not pretty or advanced, it worked. I am very proud. 

Thus ends my speech... please enjoy the code!

In [1]:
# Imports for codes. 
#At this point not everything here was used but this is all then nonsense 
#I tried before coming up with the results
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import io
import csv
from zipfile import ZipFile
from itertools import islice
import pandas as pd
from pandas import DataFrame
import pandas_gbq

In [16]:
%load_ext google.cloud.bigquery

In [2]:
# Creating paths to my files and json access document. 
service_path = "/Users/kayleeosentowski/Desktop/MSBA/ADA_21/Wedge/"
service_file = 'osentowski-project-3a5d8e60c1a3.json' # authentication information  
gbq_proj_id = 'osentowski-project' # GBQ project 
gbq_dataset_id = 'Transaction' 

private_key =service_path + service_file

In [4]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [6]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## Building Tables in QBQ

In [7]:
#This is naming the table for GBQ
my_table = "transArchive_201701"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

In [8]:
#Looking to see if the table already exists.
def tbl_exists(client, table_ref):
    from google.cloud.exceptions import NotFound
    try:
        client.get_table(table_ref)
        return True
    except NotFound:
        return False

In [9]:
#If the table doesn't exist, build one
if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name)

In [10]:
#Showing that the table now is in GBQ but is empty (0 columns)
table = client.get_table(table_ref)
print("Table {} contains {} columns".format(table_ref.table_id,len(table.schema)))

Table transArchive_201701 contains 0 columns


In [11]:
#Now we are adding the details to the empty table in GBQ. 
#The below work is to give positions to create schema and write in the table
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
job_config.schema_update_options = [
    bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION # This allows us to modify the table. 
]

In [12]:
job_config.schema = [
    bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Scale", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE"),
]
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1

## Adding data to GBQ tables

In [13]:
#Opening files (this is the last one I did)
with open("transArchive_201701_clean.csv", "rb") as source_file:
    job = client.load_table_from_file(
        source_file,
        table_ref,
        location="US",  # Must match the destination dataset location.
        job_config=job_config,
    )  # API request

In [14]:
job.result()  # Waits for table load to complete.
print(
    "Loaded {} rows into {}:{}.".format(job, 'Transaction', table_ref.table_id
    )
)

# Checks the updated length of the schema
table = client.get_table(table)
print("Table {} now contains {} columns.".format(table_ref.table_id, len(table.schema)))

Loaded LoadJob<project=osentowski-project, location=US, id=496ce4a7-9776-48bc-b686-7998baa9388f> rows into Transaction:transArchive_201701.
Table transArchive_201701 now contains 50 columns.


## Checking for success

In [25]:
# Let's query to see how many owners and records we now have loaded in

qry = """SELECT
              COUNT(DISTINCT card_no) as Owners,
              COUNT(*) as Records
         FROM  
             `osentowski-project.Transaction.transArchive*`"""

# And we execute queries with `client.query`
query_job = client.query(
    qry,
    location="US",
)

for row in query_job :
    print(f"We have {row[0]} owners and {row[1]} total records.")



We have 27208 owners and 87561182 total records.


## Conclusion

Victory! 27,208 owners and 87,561,182 total records!